<h1>Shor Algorithm via Q# and Jupyter Notebook</h1>


<c>Seong Cho</c>

This is an implementation of Shor Algorithm using Q# and Jupyter Notebook. 

<h3>1. Rotation</h3>

    This is a custom operation that would do a rotation with two qubits, one control qubit and one target qubit. 

In [2]:
operation ControlledRotation (control : Qubit, target :Qubit, theta : Double) : Unit is Adj {
    Controlled Rx([control], (theta, target));
}

ControlledRotation

<h3>2. Algorithm</h3>

   The main algorithm. It takes an array of 5 qubits, run them through the Shor circuit, measure and reset, and return a binary integer. 

In [3]:
    operation RunShor(qs: Qubit[]): Int {        
    
        //if the array of qubits has less than 5, it cannot proceed further. Return a default value.  
        if (Length(qs) < 5) {
            return 0;
        }       
        
        
        //add hadamard gates on first 3 qubits so that they would be phased. 
        H(qs[0]);
        H(qs[1]);
        H(qs[2]);
        
        //entangle them with two control qubits        
        CNOT(qs[2], qs[3]);
        CNOT(qs[2], qs[4]);
        
        
        //hadamard on 2nd qubit. 
        H(qs[1]);
        
        //1/2 pi rotation with 1st and 2nd
        ControlledRotation(qs[0], qs[1], 1.57079632679);
        
        //hadamard on 1st qubit
        H(qs[0]);
        
        //1/4 pi rotation with 2nd and 3rd
        ControlledRotation(qs[1], qs[2], 0.78539816339);
        
        //1/2 pi rotation on 1st and 3red
        ControlledRotation(qs[0], qs[2], 1.57079632679);
        
    
        //measure and write out the binary results from first three qubits        
        let outcome0 = (M(qs[0]) == One ? 0b1 | 0);
        let outcome1 = (M(qs[1]) == One ? 0b10 | 0);
        let outcome2 = (M(qs[2]) == One ? 0b100 | 0);

        //reset the qubits for next iteration        
        for index in 0 .. Length(qs) - 1 {
            Reset(qs[index]);
        }

        return outcome0 + outcome1 + outcome2; 
    }

RunShor

<h3>3. Loop and Output</h3>

    This send the request to the main algorithm as many times as required to get a statistical result. 

In [4]:
    operation Loop (count :Int) : Unit {


        mutable reg000 = 0;
        mutable reg001 = 0;
        mutable reg010 = 0;
        mutable reg011 = 0;
        mutable reg100 = 0;
        mutable reg101 = 0;
        mutable reg111 = 0;          
        mutable regerr = 0;
   
    
        use qs = Qubit[5];
   
        for index in 0 .. count {
        
        
            let shorResult = RunShor(qs);
    
            if (shorResult == 0) {
                set reg000 +=1;
            }
        
            elif (shorResult == 0b1) {
                set reg001 +=1;
            }
        
            elif (shorResult == 0b10) {
                set reg010 += 1;
            }
        
            elif (shorResult == 0b11) {
                set reg011 += 1;
            }
        
            elif (shorResult == 0b100) {
                set reg100 += 1;
            }
        
            elif (shorResult == 0b101) {
                set reg101 += 1;
            }
        
            elif (shorResult == 0b111) {
                set reg111 += 1;
            }
        
            else {
                set regerr += 1;
            }
        }
    
    
        //output the iterated results
        
        Message($"Total: {count}");
        Message($"|>000 : {reg000}");
        Message($"|>001 : {reg001}");
        Message($"|>010 : {reg010}");
        Message($"|>011 : {reg011}");
        Message($"|>100 : {reg100}");
        Message($"|>101 : {reg101}");
        Message($"|>111 : {reg111}");
    
        Message($"Errors : {regerr}");
    
}

Loop

<h3> 4. Entry point</h3>

    This is the entry point, which would be a one liner. 

In [6]:
   operation Main(): Unit {
       Loop(2048);
   }

Main

<h3>5. Simulation</h3>

   Run the simulation of the entire code

In [7]:
%simulate Main

Total: 2048
|>000 : 795
|>001 : 24
|>010 : 131
|>011 : 130
|>100 : 730
|>101 : 25
|>111 : 117
Errors : 97


()